In [2]:
# understand tensors
import tensorflow as tf

tf.reset_default_graph()

# Define a placeholder
a = tf.placeholder("float", name='pholdA')  
print("a:", a)

# Define a variable 
b = tf.Variable(2.0, name='varB')  
print("b:", b)

# Define a constant
c = tf.constant([1., 2., 3., 4.], name='consC')  
print("c:", c)  

a: Tensor("pholdA:0", dtype=float32)
b: <tf.Variable 'varB:0' shape=() dtype=float32_ref>
c: Tensor("consC:0", shape=(4,), dtype=float32)


In [3]:
d = a * b + c
d

<tf.Tensor 'add:0' shape=<unknown> dtype=float32>

In [4]:
# call the default graph
graph = tf.get_default_graph()

# print operations in the graph
for op in graph.get_operations():  
    print(op.name)

pholdA
varB/initial_value
varB
varB/Assign
varB/read
consC
mul
add
